<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci517-rmaus-fall24/blob/main/project-part-3/notebooks/cleaning_statistical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [54]:
import pandas as pd
import statsmodels.api as sm
from scipy import stats
import itertools
from statsmodels.formula.api import ols

# Data Preprocessing for Statistical Analysis

In [20]:
url = 'https://docs.google.com/spreadsheets/d/1gIxdXSMysCDWhsoB65k2T6YuW4eKZefjCap3nMCWiXM/export?format=csv'
df = pd.read_csv(url)
df.tail()

,Timestamp,Column 1,Age,Work/Study Hours per Day,"Overall Productivity Rating\nOn a scale of 1 to 5, how would you rate your daily productivity?","How consistent is your productivity throughout the day?\nOn a scale of 1 to 5, how would you rate your productivity consistency?",Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Early Morning],Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Morning],Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Afternoon],Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Evening],Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Late Night],"Frequency of Caffeinated Beverage Consumption\nHow many cups (6 - 8 oz) of caffeinated beverages (coffee, tea, energy drinks) do you consume in a day?\n(Enter a valid number between 0 to 10)",At what time(s) of day do you usually consume caffeinated beverages? \n(Select all that apply),Daily Hours of Sleep\nHow many hours do you sleep daily? \n(Enter a valid number between 0 to 16),"Do you have a consistent sleep schedule \n(i.e., going to bed and waking up at the same times each day)?","Frequency of Physical Exercise \nOn a scale of 0 to 7, how often do you engage in physical exercise per week?","Energy level throughout the day\nDuring the day do you consistently notice fluctuations in your energy levels, such as times of higher or lower energy?",What time of day do you usually engage in physical activity?,Email Address
39,12/2/2024 13:24:48,I Accept,21,4,4,4,3,5,4,4,2,1.0,"Morning, Evening",7,4,4,4 - High energy for most of the day,Morning,NaN
40,12/2/2024 13:37:37,I Accept,25,3,3,3,4,4,4,2,2,1.0,Morning,7,4,6,3 - Energy constant/neutral at any given time ...,Morning,NaN
41,12/2/2024 15:55:23,I Accept,24,3,2,3,3,3,2,2,1,1.0,Morning,7,2,1,2 - Low energy for most of the day,Morning,NaN
42,12/2/2024 18:29:28,I Accept,26,8,4,3,5,5,3,3,5,1.0,Morning,7,3,7,3 - Energy constant/neutral at any given time ...,"Early Morning, Morning",NaN
43,12/3/2024 23:56:45,I Accept,24,10,4,3,4,5,3,2,4,3.0,Evening,8,3,3,3 - Energy constant/neutral at any given time ...,"Early Morning, Evening",NaN


# Clean data

In [21]:
df.columns

Index(['Timestamp', 'Column 1', 'Age', 'Work/Study Hours per Day',
       'Overall Productivity Rating\nOn a scale of 1 to 5, how would you rate your daily productivity?',
       'How consistent is your productivity throughout the day?\nOn a scale of 1 to 5, how would you rate your productivity consistency?',
       'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Early Morning]',
       'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Morning]',
       'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Afternoon]',
       'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Evening]',
       'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Prod

In [27]:
def clean_data(df):
    # Make a copy of the dataframe
    df_clean = df.copy()

    new_columns = {
        'Timestamp': 'timestamp',
        'Column 1': 'consent',
        'Age': 'age',
        'Work/Study Hours per Day': 'work_study_hours',
        'Overall Productivity Rating\nOn a scale of 1 to 5, how would you rate your daily productivity?': 'daily_productivity',
        'How consistent is your productivity throughout the day?\nOn a scale of 1 to 5, how would you rate your productivity consistency?': 'productivity_consistency',
        'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Early Morning]': 'early_morning_prod',
        'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Morning]': 'morning_prod',
        'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Afternoon]': 'afternoon_prod',
        'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Evening]': 'evening_prod',
        'Productivity levels throughout the day\nOn the scale of 1 to 5 rate yourself\n1 - Least Productive\n5 - Most Productive [Late Night]': 'late_night_prod',
        'Frequency of Caffeinated Beverage Consumption\nHow many cups (6 - 8 oz) of caffeinated beverages (coffee, tea, energy drinks) do you consume in a day?\n(Enter a valid number between 0 to 10)': 'caffeine_consumption',
        'At what time(s) of day do you usually consume caffeinated beverages? \n(Select all that apply)': 'caffeine_timing',
        'Daily Hours of Sleep\nHow many hours do you sleep daily? \n(Enter a valid number between 0 to 16)': 'sleep_hours',
        'Do you have a consistent sleep schedule \n(i.e., going to bed and waking up at the same times each day)?': 'consistent_sleep',
        'Frequency of Physical Exercise \nOn a scale of 0 to 7, how often do you engage in physical exercise per week?': 'exercise_freq',
        'Energy level throughout the day\nDuring the day do you consistently notice fluctuations in your energy levels, such as times of higher or lower energy?  ': 'energy_fluctuations',
        'What time of day do you usually engage in physical activity?': 'exercise_timing',
        'Email Address': 'email'
    }

    df_clean.rename(columns=new_columns, inplace=True)
    df_clean.drop(columns=["timestamp", "consent", "email"], inplace=True)

    # Handle missing or invalid values
    df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
    df_clean['work_study_hours'] = pd.to_numeric(df_clean['work_study_hours'], errors='coerce')
    df_clean['daily_productivity'] = pd.to_numeric(df_clean['daily_productivity'], errors='coerce')
    df_clean['productivity_consistency'] = pd.to_numeric(df_clean['productivity_consistency'], errors='coerce')
    df_clean['sleep_hours'] = pd.to_numeric(df_clean['sleep_hours'], errors='coerce')
    df_clean['caffeine_consumption'] = pd.to_numeric(df_clean['caffeine_consumption'], errors='coerce')
    df_clean['exercise_freq'] = pd.to_numeric(df_clean['exercise_freq'], errors='coerce')

    # Clean Energy Level - extract numeric value
    df_clean['energy_fluctuations'] = df_clean['energy_fluctuations'].str.extract('(\d+)').astype(int)

    # Create dummy variables for time-based categories
    caffeine_dummies = df_clean['caffeine_timing'].str.get_dummies(sep=', ')
    exercise_dummies = df_clean['exercise_timing'].str.get_dummies(sep=', ')

    # Standardize column names: lowercase and prefix with context
    caffeine_dummies.columns = [f'caffeine_{col.lower().replace(" ", "_")}' for col in caffeine_dummies.columns]
    exercise_dummies.columns = [f'exercise_{col.lower().replace(" ", "_")}' for col in exercise_dummies.columns]

    # Concatenate dummy variables and drop original columns
    df_clean = pd.concat([df_clean, caffeine_dummies, exercise_dummies], axis=1)
    df_clean.drop(columns=['caffeine_timing', 'exercise_timing'], inplace=True)

    return df_clean

df_clean = clean_data(df)
df_clean.head()

,age,work_study_hours,daily_productivity,productivity_consistency,early_morning_prod,morning_prod,afternoon_prod,evening_prod,late_night_prod,caffeine_consumption,...,caffeine_afternoon,caffeine_evening,caffeine_late_night,caffeine_morning,caffeine_never,exercise_afternoon,exercise_early_morning,exercise_evening,exercise_morning,exercise_never
0,40,12,4,4,4,4,4,3,2,3.0,...,0,0,0,1,0,1,0,0,0,0
1,24,1,1,2,1,1,1,1,1,0.0,...,0,0,0,0,1,0,0,1,0,0
2,23,10,4,4,4,5,2,5,2,1.5,...,0,1,0,1,0,1,0,0,0,0
3,24,8,4,3,2,3,5,4,5,0.0,...,0,0,0,0,1,1,0,1,0,0
4,23,8,5,5,5,5,5,5,5,1.0,...,0,1,0,1,0,0,1,1,1,0


In [28]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       44 non-null     int64  
 1   work_study_hours          44 non-null     int64  
 2   daily_productivity        44 non-null     int64  
 3   productivity_consistency  44 non-null     int64  
 4   early_morning_prod        44 non-null     int64  
 5   morning_prod              44 non-null     int64  
 6   afternoon_prod            44 non-null     int64  
 7   evening_prod              44 non-null     int64  
 8   late_night_prod           44 non-null     int64  
 9   caffeine_consumption      44 non-null     float64
 10  sleep_hours               44 non-null     int64  
 11  consistent_sleep          44 non-null     int64  
 12  exercise_freq             44 non-null     int64  
 13  energy_fluctuations       44 non-null     int64  
 14  caffeine_aft

### **Column Classification**
|Columns | Type |
|-|-|
|`age`| Appears numeric but is often treated as a discrete variable.|
|`daily_productivity`, `productivity_consistency`, `early_morning_prod`, `morning_prod`, `afternoon_prod`, `evening_prod`, `late_night_prod`| Ordinal data (Likert scale)|
|`work_study_hours`, `caffeine_consumption`, `sleep_hours`, `consistent_sleep`, `exercise_freq`, `energy_fluctuations`| Discrete numeric|
|`caffeine_*`, `exercise_*`| Binary discrete variables (One-hot encoded)|


# Analysis

In [43]:
df_clean.columns

Index(['age', 'work_study_hours', 'daily_productivity',
       'productivity_consistency', 'early_morning_prod', 'morning_prod',
       'afternoon_prod', 'evening_prod', 'late_night_prod',
       'caffeine_consumption', 'sleep_hours', 'consistent_sleep',
       'exercise_freq', 'energy_fluctuations', 'caffeine_afternoon',
       'caffeine_evening', 'caffeine_late_night', 'caffeine_morning',
       'caffeine_never', 'exercise_afternoon', 'exercise_early_morning',
       'exercise_evening', 'exercise_morning', 'exercise_never'],
      dtype='object')

## Multiple Linear Regression Analysis

In [30]:
numerical_columns = df_clean.select_dtypes(include=['int64']).columns
numerical_columns

Index(['age', 'work_study_hours', 'daily_productivity',
       'productivity_consistency', 'early_morning_prod', 'morning_prod',
       'afternoon_prod', 'evening_prod', 'late_night_prod', 'sleep_hours',
       'consistent_sleep', 'exercise_freq', 'energy_fluctuations',
       'caffeine_afternoon', 'caffeine_evening', 'caffeine_late_night',
       'caffeine_morning', 'caffeine_never', 'exercise_afternoon',
       'exercise_early_morning', 'exercise_evening', 'exercise_morning',
       'exercise_never'],
      dtype='object')

In [31]:
df_numerical = df_clean[numerical_columns]
df_numerical.head()

,age,work_study_hours,daily_productivity,productivity_consistency,early_morning_prod,morning_prod,afternoon_prod,evening_prod,late_night_prod,sleep_hours,...,caffeine_afternoon,caffeine_evening,caffeine_late_night,caffeine_morning,caffeine_never,exercise_afternoon,exercise_early_morning,exercise_evening,exercise_morning,exercise_never
0,40,12,4,4,4,4,4,3,2,7,...,0,0,0,1,0,1,0,0,0,0
1,24,1,1,2,1,1,1,1,1,10,...,0,0,0,0,1,0,0,1,0,0
2,23,10,4,4,4,5,2,5,2,7,...,0,1,0,1,0,1,0,0,0,0
3,24,8,4,3,2,3,5,4,5,9,...,0,0,0,0,1,1,0,1,0,0
4,23,8,5,5,5,5,5,5,5,7,...,0,1,0,1,0,0,1,1,1,0


In [32]:
# DV - daily_productivity, and productivity_consistency
X = df_numerical.drop(columns=['daily_productivity', 'productivity_consistency'])  # Features
y1 = df_numerical['daily_productivity']
y2 = df_numerical['productivity_consistency']

# Add constant for statsmodels
X = sm.add_constant(X)

model1 = sm.OLS(y1, X).fit()
model2 = sm.OLS(y2, X).fit()

summary1 = model1.summary()
summary2 = model2.summary()

# Without Scaling

### y1 column is 'productivity_rating'

In [33]:
print(summary1)

                            OLS Regression Results                            
Dep. Variable:     daily_productivity   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     4.379
Date:                Wed, 04 Dec 2024   Prob (F-statistic):           0.000542
Time:                        09:00:15   Log-Likelihood:                -15.430
No. Observations:                  44   AIC:                             74.86
Df Residuals:                      22   BIC:                             114.1
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

### y2 column is 'productivity_consistency'

In [56]:
print(summary2)

                               OLS Regression Results                               
Dep. Variable:     productivity_consistency   R-squared:                       0.674
Model:                                  OLS   Adj. R-squared:                  0.363
Method:                       Least Squares   F-statistic:                     2.166
Date:                      Wed, 04 Dec 2024   Prob (F-statistic):             0.0393
Time:                              09:00:15   Log-Likelihood:                -30.687
No. Observations:                        44   AIC:                             105.4
Df Residuals:                            22   BIC:                             144.6
Df Model:                                21                                         
Covariance Type:                  nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

# With Scaling

In [35]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X.drop('const', axis=1))
X_scaled = sm.add_constant(X_scaled)

### y1 column is 'productivity_rating'

In [36]:
model_scaled_1 = sm.OLS(y1, X_scaled).fit()
print(model_scaled_1.summary())

                            OLS Regression Results                            
Dep. Variable:     daily_productivity   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     4.379
Date:                Wed, 04 Dec 2024   Prob (F-statistic):           0.000542
Time:                        09:00:51   Log-Likelihood:                -15.430
No. Observations:                  44   AIC:                             74.86
Df Residuals:                      22   BIC:                             114.1
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3701      0.581      2.358      0.0

### y2 column is 'productivity_consistency'

In [37]:
model_scaled_2 = sm.OLS(y1, X_scaled).fit()
print(model_scaled_2.summary())

                            OLS Regression Results                            
Dep. Variable:     daily_productivity   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     4.379
Date:                Wed, 04 Dec 2024   Prob (F-statistic):           0.000542
Time:                        09:00:57   Log-Likelihood:                -15.430
No. Observations:                  44   AIC:                             74.86
Df Residuals:                      22   BIC:                             114.1
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3701      0.581      2.358      0.0

## Correlation Analysis

In [38]:
correlation_matrix = df_clean[numerical_columns].corr().round(3)
correlation_matrix

,age,work_study_hours,daily_productivity,productivity_consistency,early_morning_prod,morning_prod,afternoon_prod,evening_prod,late_night_prod,sleep_hours,...,caffeine_afternoon,caffeine_evening,caffeine_late_night,caffeine_morning,caffeine_never,exercise_afternoon,exercise_early_morning,exercise_evening,exercise_morning,exercise_never
age,1.000,0.401,0.096,0.015,-0.131,0.048,0.252,-0.183,-0.066,-0.175,...,0.244,-0.031,-0.205,0.225,-0.157,0.221,-0.158,0.079,-0.188,-0.061
work_study_hours,0.401,1.000,0.529,0.233,-0.015,0.127,0.309,0.198,0.217,-0.317,...,-0.118,0.247,-0.196,0.211,-0.134,0.123,0.157,0.124,-0.291,-0.057
daily_productivity,0.096,0.529,1.000,0.574,0.482,0.316,0.398,0.475,0.298,-0.436,...,-0.208,0.324,-0.157,0.174,-0.137,0.108,0.103,0.053,0.134,-0.117
productivity_consistency,0.015,0.233,0.574,1.000,0.404,0.222,0.311,0.350,-0.039,-0.193,...,-0.235,-0.004,0.055,-0.080,0.136,0.237,-0.157,0.044,0.091,-0.151
early_morning_prod,-0.131,-0.015,0.482,0.404,1.000,0.510,-0.053,0.090,-0.183,-0.321,...,-0.371,0.153,-0.032,-0.018,0.129,0.115,0.152,-0.131,0.110,0.036
morning_prod,0.048,0.127,0.316,0.222,0.510,1.000,0.340,0.071,-0.146,-0.373,...,-0.021,0.359,0.034,0.169,-0.211,0.173,0.169,-0.139,0.002,0.178
afternoon_prod,0.252,0.309,0.398,0.311,-0.053,0.340,1.000,0.060,0.096,-0.248,...,0.162,0.100,-0.061,0.210,-0.285,0.217,-0.069,0.287,-0.028,-0.036
evening_prod,-0.183,0.198,0.475,0.350,0.090,0.071,0.060,1.000,0.631,-0.318,...,-0.158,0.299,0.099,0.127,-0.016,0.084,-0.229,0.066,0.130,-0.091
late_night_prod,-0.066,0.217,0.298,-0.039,-0.183,-0.146,0.096,0.631,1.000,-0.164,...,0.060,0.239,-0.092,0.110,-0.083,-0.227,0.070,0.279,-0.046,-0.039
sleep_hours,-0.175,-0.317,-0.436,-0.193,-0.321,-0.373,-0.248,-0.318,-0.164,1.000,...,0.058,-0.334,0.024,-0.219,0.203,0.036,0.063,-0.147,-0.090,0.211


In [40]:
import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns.tolist(),
    y=correlation_matrix.index.tolist(),
    colorscale='Teal',
    showscale=True
)

fig.update_layout(
    title='Correlation Matrix of Productivity Factors',
    margin=dict(t=200),  #t-top
    width=1200,
    height=1000
)

fig.update_traces(showscale=True, colorbar=dict(thickness=20, len=0.8))
fig.update_layout(template="plotly_dark")
fig.show()

## 1-way ANOVA

In [61]:
# Conduct one-way ANOVA for all caffeine-related dummy variables against daily_productivity
for col in df_clean.filter(like='caffeine_').columns:
    f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['daily_productivity'],
                                  df_clean[df_clean[col] == 0]['daily_productivity'])
    print(f"One-way ANOVA for {col} vs daily_productivity: F={f_val:.2f}, p={p_val:.4f}")

# Conduct one-way ANOVA for all exercise-related dummy variables against daily_productivity
for col in df_clean.filter(like='exercise_').columns:
    f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['daily_productivity'],
                                  df_clean[df_clean[col] == 0]['daily_productivity'])
    print(f"One-way ANOVA for {col} vs daily_productivity: F={f_val:.2f}, p={p_val:.4f}")

print("\n", "#"*90, "\n")

# Conduct one-way ANOVA for all caffeine-related dummy variables against productivity_consistency
for col in df_clean.filter(like='caffeine_').columns:
    f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['productivity_consistency'],
                                  df_clean[df_clean[col] == 0]['productivity_consistency'])
    print(f"One-way ANOVA for {col} vs productivity_consistency: F={f_val:.2f}, p={p_val:.4f}")

# Conduct one-way ANOVA for all exercise-related dummy variables against productivity_consistency
for col in df_clean.filter(like='exercise_').columns:
    f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['productivity_consistency'],
                                  df_clean[df_clean[col] == 0]['productivity_consistency'])
    print(f"One-way ANOVA for {col} vs productivity_consistency: F={f_val:.2f}, p={p_val:.4f}")


# # Initialize lists to store results
# results = []

# # Conduct one-way ANOVA for all caffeine-related dummy variables against daily_productivity
# for col in df_clean.filter(like='caffeine_').columns:
#     f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['daily_productivity'],
#                                   df_clean[df_clean[col] == 0]['daily_productivity'])
#     results.append({'Variable': col, 'Dependent Variable': 'daily_productivity', 'F-value': f_val, 'p-value': p_val})

# # Conduct one-way ANOVA for all caffeine-related dummy variables against productivity_consistency
# for col in df_clean.filter(like='caffeine_').columns:
#     f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['productivity_consistency'],
#                                   df_clean[df_clean[col] == 0]['productivity_consistency'])
#     results.append({'Variable': col, 'Dependent Variable': 'productivity_consistency', 'F-value': f_val, 'p-value': p_val})

# # Conduct one-way ANOVA for all exercise-related dummy variables against daily_productivity
# for col in df_clean.filter(like='exercise_').columns:
#     f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['daily_productivity'],
#                                   df_clean[df_clean[col] == 0]['daily_productivity'])
#     results.append({'Variable': col, 'Dependent Variable': 'daily_productivity', 'F-value': f_val, 'p-value': p_val})

# # Conduct one-way ANOVA for all exercise-related dummy variables against productivity_consistency
# for col in df_clean.filter(like='exercise_').columns:
#     f_val, p_val = stats.f_oneway(df_clean[df_clean[col] == 1]['productivity_consistency'],
#                                   df_clean[df_clean[col] == 0]['productivity_consistency'])
#     results.append({'Variable': col, 'Dependent Variable': 'productivity_consistency', 'F-value': f_val, 'p-value': p_val})

# # Convert results to a DataFrame and display
# results_df = pd.DataFrame(results)
# print("\nOne-way ANOVA Results:")
# results_df

One-way ANOVA for caffeine_consumption vs daily_productivity: F=0.88, p=0.3554
One-way ANOVA for caffeine_afternoon vs daily_productivity: F=1.90, p=0.1752
One-way ANOVA for caffeine_evening vs daily_productivity: F=4.94, p=0.0318
One-way ANOVA for caffeine_late_night vs daily_productivity: F=1.06, p=0.3081
One-way ANOVA for caffeine_morning vs daily_productivity: F=1.32, p=0.2576
One-way ANOVA for caffeine_never vs daily_productivity: F=0.81, p=0.3741
One-way ANOVA for exercise_freq vs daily_productivity: F=6.89, p=0.0393
One-way ANOVA for exercise_afternoon vs daily_productivity: F=0.49, p=0.4862
One-way ANOVA for exercise_early_morning vs daily_productivity: F=0.45, p=0.5069
One-way ANOVA for exercise_evening vs daily_productivity: F=0.12, p=0.7323
One-way ANOVA for exercise_morning vs daily_productivity: F=0.77, p=0.3867
One-way ANOVA for exercise_never vs daily_productivity: F=0.58, p=0.4512

 ########################################################################################

## 2-way ANOVA

In [68]:
# Ensure that categorical columns are properly encoded
caffeine_vars = df_clean.filter(like='caffeine_').columns
exercise_vars = df_clean.filter(like='exercise_').columns

# Convert caffeine and exercise columns to categorical data types if not already
df_clean[caffeine_vars] = df_clean[caffeine_vars].astype('category')
df_clean[exercise_vars] = df_clean[exercise_vars].astype('category')

# Two-way ANOVA for daily_productivity
formula_daily_productivity = 'daily_productivity ~ C(caffeine_afternoon) + C(caffeine_evening) + C(caffeine_late_night) + C(caffeine_morning) + C(caffeine_never) + C(exercise_afternoon) + C(exercise_early_morning) + C(exercise_evening) + C(exercise_morning) + C(exercise_never) + C(caffeine_afternoon):C(exercise_afternoon) + C(caffeine_evening):C(exercise_evening) + C(caffeine_morning):C(exercise_morning) + C(caffeine_never):C(exercise_never)'
model_daily_productivity = ols(formula_daily_productivity, data=df_clean).fit()
anova_table_daily_productivity = sm.stats.anova_lm(model_daily_productivity, type=2)
print("\nTwo-way ANOVA for caffeine and exercise on daily_productivity:")
print(anova_table_daily_productivity)

# Two-way ANOVA for productivity_consistency
formula_productivity_consistency = 'productivity_consistency ~ C(caffeine_afternoon) + C(caffeine_evening) + C(caffeine_late_night) + C(caffeine_morning) + C(caffeine_never) + C(exercise_afternoon) + C(exercise_early_morning) + C(exercise_evening) + C(exercise_morning) + C(exercise_never) + C(caffeine_afternoon):C(exercise_afternoon) + C(caffeine_evening):C(exercise_evening) + C(caffeine_morning):C(exercise_morning) + C(caffeine_never):C(exercise_never)'
model_productivity_consistency = ols(formula_productivity_consistency, data=df_clean).fit()
anova_table_productivity_consistency = sm.stats.anova_lm(model_productivity_consistency, type=2)
print("\nTwo-way ANOVA for caffeine and exercise on productivity_consistency:")
print(anova_table_productivity_consistency)


Two-way ANOVA for caffeine and exercise on daily_productivity:
                                               df     sum_sq   mean_sq  \
C(caffeine_afternoon)                         1.0   1.165501  1.165501   
C(caffeine_evening)                           1.0   2.578612  2.578612   
C(caffeine_late_night)                        1.0   0.900995  0.900995   
C(caffeine_morning)                           1.0   0.280056  0.280056   
C(caffeine_never)                             1.0   0.027501  0.027501   
C(exercise_afternoon)                         1.0   0.493474  0.493474   
C(exercise_early_morning)                     1.0   0.140808  0.140808   
C(exercise_evening)                           1.0   0.430422  0.430422   
C(exercise_morning)                           1.0   1.702247  1.702247   
C(exercise_never)                             1.0   0.011735  0.011735   
C(caffeine_afternoon):C(exercise_afternoon)   1.0   1.002884  1.002884   
C(caffeine_evening):C(exercise_evening)       1.

## Chi-square contingency

In [70]:
# List of caffeine and exercise variables
caffeine_vars = df_clean.filter(like='caffeine_').columns
exercise_vars = df_clean.filter(like='exercise_').columns

# DataFrame to store results
results = []

# Function to run chi-square tests and store results
def chi_square_test(data, var1, var2):
    # Create a contingency table
    contingency_table = pd.crosstab(data[var1], data[var2])
    # Run chi-square test
    chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)
    return chi2_stat, p_value, dof, expected

# Run chi-square tests for all combinations and store in the results DataFrame
for col1 in caffeine_vars:
    for col2 in exercise_vars:
        chi2_stat, p_value, dof, expected = chi_square_test(df_clean, col1, col2)
        results.append({
            'Caffeine Variable': col1,
            'Exercise Variable': col2,
            'Chi2 Stat': round(chi2_stat, 2),
            'p-value': round(p_value, 4),
            'Degrees of Freedom': dof
        })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results as a table
print("Chi-Square Test Results:")
results_df

Chi-Square Test Results:


,Caffeine Variable,Exercise Variable,Chi2 Stat,p-value,Degrees of Freedom
0,caffeine_consumption,exercise_freq,22.81,0.9439,35
1,caffeine_consumption,exercise_afternoon,12.72,0.0261,5
2,caffeine_consumption,exercise_early_morning,1.71,0.8871,5
3,caffeine_consumption,exercise_evening,5.34,0.3758,5
4,caffeine_consumption,exercise_morning,2.39,0.7930,5
5,caffeine_consumption,exercise_never,0.91,0.9697,5
6,caffeine_afternoon,exercise_freq,6.52,0.4804,7
7,caffeine_afternoon,exercise_afternoon,0.06,0.8013,1
8,caffeine_afternoon,exercise_early_morning,0.00,1.0000,1
9,caffeine_afternoon,exercise_evening,0.05,0.8284,1


In [75]:
# Assuming daily_productivity and productivity_consistency are categorized as 'low', 'medium', 'high'

# Convert daily_productivity and productivity_consistency to categorical variables (optional)
df_clean['daily_productivity_cat'] = pd.qcut(df_clean['daily_productivity'], q=3, labels=['low', 'medium', 'high'], duplicates='drop')
df_clean['productivity_consistency_cat'] = pd.cut(df_clean['productivity_consistency'], bins=3, labels=['low', 'medium', 'high'])

# List of variables to test against
productivity_vars = ['daily_productivity_cat', 'productivity_consistency_cat']

# Run chi-square tests for each productivity variable with caffeine and exercise variables
results = []

for prod_var in productivity_vars:
    for col1 in caffeine_vars:
        for col2 in exercise_vars:
            chi2_stat, p_value, dof, expected = chi_square_test(df_clean, prod_var, col1)
            results.append({
                'Productivity Variable': prod_var,
                'Variable': col1,
                'Chi2 Stat': round(chi2_stat, 2),
                'p-value': round(p_value, 4),
                'Degrees of Freedom': dof
            })
            chi2_stat, p_value, dof, expected = chi_square_test(df_clean, prod_var, col2)
            results.append({
                'Productivity Variable': prod_var,
                'Variable': col2,
                'Chi2 Stat': round(chi2_stat, 2),
                'p-value': round(p_value, 4),
                'Degrees of Freedom': dof
            })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
print("Chi-Square Test Results Related to Productivity Measures:")
results_df

Chi-Square Test Results Related to Productivity Measures:


,Productivity Variable,Variable,Chi2 Stat,p-value,Degrees of Freedom
0,daily_productivity_cat,caffeine_consumption,9.38,0.4962,10
1,daily_productivity_cat,exercise_freq,27.86,0.0149,14
2,daily_productivity_cat,caffeine_consumption,9.38,0.4962,10
3,daily_productivity_cat,exercise_afternoon,1.14,0.5657,2
4,daily_productivity_cat,caffeine_consumption,9.38,0.4962,10
...,...,...,...,...,...
139,productivity_consistency_cat,exercise_evening,5.08,0.0787,2
140,productivity_consistency_cat,caffeine_never,1.31,0.5196,2
141,productivity_consistency_cat,exercise_morning,0.86,0.6517,2
142,productivity_consistency_cat,caffeine_never,1.31,0.5196,2


# THE END